<a href="https://colab.research.google.com/github/bamacgabhann/GY5021/blob/2024/GY5021/1_Introduction_to_Geospatial_Data/GY5021_5_Coordinate_Reference_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/bamacgabhann/GY5021/2024/PD_logo.png" align=center alt="UL Geography logo"/>

# Coordinate Reference Systems

In the Vector Data and Attribute Data notebooks, we looked at some examples of vector data with points, lines, and polygons - but most of the examples used simple made-up integer coordinates. But we also used one example in the Attribute Data notebook which used coordinates looking more real, each with two six-figure numbers. But what type of coordinates are these?

Of course, this raises some further questions. Are there different kinds of coordinates? How can we tell the difference? And how do we make sure that all our data is using the same type of coordinates, or at least is plotted together in the right place on a map?

To explore and understand this, first let's look at how maps of the world are made.

## 7. Coordinate Reference Systems

pyproj is a library which is designed to handle coordinate reference systems. 

<img src="https://pyproj4.github.io/pyproj/stable/_static/logo.png" style="height:60px" alt="pyproj logo"/>

Like NumPy, it is written in C, so although it's called from python, all the computation is done at a compiled level. 

pyproj can interpret references to a CRS in numerous different formats. From the pyproj documentation (https://pyproj4.github.io/pyproj/stable/api/crs/crs.html#pyproj.crs.CRS.from_user_input):

 -  PROJ string  
 - Dictionary of PROJ parameters 
 - PROJ keyword arguments for parameters  
 - JSON string with PROJ parameters  
 - CRS WKT string  
 - An authority string, e.g. ```'epsg:4326'```  
 - An EPSG integer e.g. ```'4326'```  
 - A tuple of (“auth_name”: “auth_e.g.”) e.g. ```('epsg',   '4326')```  
 - An object with a ```to_wkt``` method  
 - A ```pyproj.crs.CRS``` class 

pyproj allows for the definition of custom coordinate reference systems, and as per the list above, can essentially handle any pre-defined crs. It also facilitates transformations between coordinate reference systems. Using the above example:

In [24]:
gdf = gdf.to_crs(4326)
gdf

,Meas_time,PM2.5,PM10,Pressure,Temperature,Humidity,geometry
0,2023-11-01 08:00:00,3,5,1012,12,86,POINT (-8.66229 52.63825)
1,2023-11-01 08:00:30,4,6,1012,12,86,POINT (-8.66229 52.63825)
2,2023-11-01 08:01:00,6,5,1012,13,87,POINT (-8.66229 52.63825)
3,2023-11-01 08:01:30,2,3,1013,13,87,POINT (-8.66229 52.63825)
4,2023-11-01 08:02:00,3,4,1013,13,87,POINT (-8.66229 52.63825)


In [25]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [26]:
gdf = gdf.to_crs('+proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs +type=crs')
gdf

,Meas_time,PM2.5,PM10,Pressure,Temperature,Humidity,geometry
0,2023-11-01 08:00:00,3,5,1012,12,86,POINT (3073629.327 3442472.493)
1,2023-11-01 08:00:30,4,6,1012,12,86,POINT (3073629.327 3442472.493)
2,2023-11-01 08:01:00,6,5,1012,13,87,POINT (3073629.327 3442472.493)
3,2023-11-01 08:01:30,2,3,1013,13,87,POINT (3073629.327 3442472.493)
4,2023-11-01 08:02:00,3,4,1013,13,87,POINT (3073629.327 3442472.493)


In [27]:
gdf.crs

<Projected CRS: +proj=laea +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3 ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Lambert Azimuthal Equal Area
Datum: Unknown based on GRS 1980 ellipsoid
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich